$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 4: Classification with QDA, LDA, and Logistic Regression

Damian Armijo

## Overview

In this assignment, you will make a new version of your ```NeuralNetwork``` class called ```NeuralNetworkClassifier```. You will then apply ```QDA```, ```LDA``` and your ```NeuralNetworkClassifier``` to a classification problem and discuss the results.  The ```tanh``` function will be used as the activation function for ```NeuralNetworkClassifier```.

### NeuralNetworkClassifier

Copy your ```neuralnetworksA2.py``` into a new file named ```neuralnetworksA4.py```.  Define a new class named ```NeuralNetworkClassifier``` that extends ```NeuralNetwork```.  The following code cell indicates which methods you must override, with comments instructing you what you must do to complete it.  Add this class to your ```neuralnetworksA4.py``` file.

In [ ]:
class NeuralNetworkClassifier(NeuralNetwork):

    def _multinomialize(self, Y):   # also known as softmax
        # fix to avoid overflow
        mx = max(0, np.max(Y))
        expY = np.exp(Y - mx)
        denom = np.sum(expY, axis=1).reshape((-1, 1)) + sys.float_info.epsilon
        return expY / denom

    def _objectiveF(self, w, X, Tindicators):
        .
        .
        .
        # Convert Y to multinomial distribution. Call result G.
        #  and return negative of mean log likelihood.
        # Write the call to np.log as  np.log(G + sys.float_info.epsilon)


    def _gradientF(self, w, X, Tindicators):
        .
        .
        .
        # Convert Y to multinomial distribution. Call result G.
        # delta calculated as before, except use Tindicators instead of T to
        #   make error of  Tindicates - G
        # Also, negate the result.  Why?
        .
        .
        .

    def train(self, X, T, nIterations=100, verbose=False,
              weightPrecision=0, errorPrecision=0, saveWeightsHistory=False):
        .
        .
        .
        # Remove T standardization calculations
        # Assign to Tindicators using ml.makeIndicatorVars
        # Add   self.classes = np.unique(T)
        # Pass Tindicators into ml.scg instead of T
        .
        .
        .

    def use(self, X, allOutputs=False):
        .
        .
        .
        # multinomialize Y, assign to G
        # Calculate predicted classes by
        #   picking argmax for each row of G
        #   and use the results to index into self.classes.
        .
        .
        .
        return (classes, G, Z[1:]) if allOutputs else classes


### Test It

In [2]:
import numpy as np
import neuralnetworksA4 as nn

SyntaxError: invalid syntax (neuralnetworksA4.py, line 203)

In [2]:
X = np.arange(10).reshape((-1, 1))
T = np.array([1]*5 + [2]*5).reshape((-1, 1))

In [3]:
netc = nn.NeuralNetworkClassifier(X.shape[1], [5, 5], len(np.unique(T)))
netc.train(X, T, 20)
print(netc)
print('T, Predicted')
print(np.hstack((T, netc.use(X))))

NeuralNetwork(1, [5, 5], 2)
   Network was trained for 21 iterations that took 0.0728 seconds. Final error is 0.007611063152805585.
T, Predicted
[[1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [2 2]
 [2 2]
 [2 2]
 [2 2]
 [2 2]]


## Addition to ```partition``` function

Add the keyword parameter ```classification``` with a default value of ```False``` to your ```partition``` function.  When its value is set to ```True``` your ```partition``` function must perform a stratified partitioning as illustrated in lecture notes [12 Introduction to Classification](http://nbviewer.ipython.org/url/www.cs.colostate.edu/~anderson/cs445/notebooks/12%20Introduction%20to%20Classification.ipynb)     

In [1]:
import mlutilities as ml
Xtrain, Ttrain, Xtest, Ttest = ml.partition(X, T, 0.6, classification=True)

ModuleNotFoundError: No module named 'matplotlib'

In [5]:
Xtrain

array([[0],
       [1],
       [2],
       [5],
       [6],
       [7]])

In [6]:
Ttrain

array([[1],
       [1],
       [1],
       [2],
       [2],
       [2]])

In [7]:
Xtest

array([[3],
       [4],
       [8],
       [9]])

In [8]:
Ttest

array([[1],
       [1],
       [2],
       [2]])

## Printing a confusion matrix

Add these two functions to your ```mlutilities.py``` file.

In [9]:
def confusionMatrix(actual, predicted, classes):
    nc = len(classes)
    confmat = np.zeros((nc, nc)) 
    for ri in range(nc):
        trues = (actual==classes[ri]).squeeze()
        predictedThisClass = predicted[trues]
        keep = trues
        predictedThisClassAboveThreshold = predictedThisClass
        # print 'confusionMatrix: sum(trues) is ', np.sum(trues),'for classes[ri]',classes[ri]
        for ci in range(nc):
            confmat[ri,ci] = np.sum(predictedThisClassAboveThreshold == classes[ci]) / float(np.sum(keep))
    printConfusionMatrix(confmat,classes)
    return confmat

def printConfusionMatrix(confmat,classes):
    print('   ',end='')
    for i in classes:
        print('%5d' % (i), end='')
    print('\n    ',end='')
    print('{:s}'.format('------'*len(classes)))
    for i,t in enumerate(classes):
        print('{:2d} |'.format(t), end='')
        for i1,t1 in enumerate(classes):
            if confmat[i,i1] == 0:
                print('  0  ',end='')
            else:
                print('{:5.1f}'.format(100*confmat[i,i1]), end='')
        print()

### Example with toy data

Use the above data to compare QDA, LDA, and linear and nonlinear logistic regression.

In [10]:
import qdalda
qda = qdalda.QDA()
qda.train(Xtrain, Ttrain)
Ytrain = qda.use(Xtrain)
Ytest = qda.use(Xtest)

In [11]:
print(np.hstack((Ttrain, Ytrain)))

[[1 1]
 [1 1]
 [1 1]
 [2 2]
 [2 2]
 [2 2]]


In [12]:
np.sum(Ttrain == Ytrain) / len(Ttrain) * 100

100.0

In [13]:
print(np.hstack((Ttest, Ytest)))

[[1 1]
 [1 2]
 [2 2]
 [2 2]]


In [14]:
np.sum(Ttest == Ytest) / len(Ttest) * 100

75.0

In [15]:
lda = qdalda.LDA()
lda.train(Xtrain, Ttrain)
Ytrain = lda.use(Xtrain)
Ytest = lda.use(Xtest)

In [16]:
print(np.hstack((Ttrain, Ytrain)))

[[1 1]
 [1 1]
 [1 1]
 [2 2]
 [2 2]
 [2 2]]


In [17]:
print(np.hstack((Ttest, Ytest)))

[[1 1]
 [1 2]
 [2 2]
 [2 2]]


In [18]:
np.sum(Ttrain == Ytrain) / len(Ttrain) * 100

100.0

In [19]:
np.sum(Ttest == Ytest) / len(Ttest) * 100

75.0

In [20]:
ml.confusionMatrix(Ttrain, Ytrain, [1, 2]);

       1    2
    ------------
 1 |100.0  0  
 2 |  0  100.0


In [21]:
ml.confusionMatrix(Ttest, Ytest, [1, 2]);

       1    2
    ------------
 1 | 50.0 50.0
 2 |  0  100.0


In [22]:
netc = nn.NeuralNetworkClassifier(X.shape[1], [5, 5], len(np.unique(T)))
netc.train(Xtrain, Ttrain, 100)
print(netc)
print('T, Predicted')
Ytrain = netc.use(Xtrain)
Ytest = netc.use(Xtest)

NeuralNetwork(1, [5, 5], 2)
   Network was trained for 101 iterations that took 0.2351 seconds. Final error is 3.041686791657381e-09.
T, Predicted


In [23]:
print(np.hstack((Ttrain, Ytrain)))

[[1 1]
 [1 1]
 [1 1]
 [2 2]
 [2 2]
 [2 2]]


In [24]:
print(np.hstack((Ttest, Ytest)))

[[1 1]
 [1 2]
 [2 2]
 [2 2]]


In [25]:
np.sum(Ttrain == Ytrain) / len(Ttrain) * 100

100.0

In [26]:
np.sum(Ttest == Ytest) / len(Ttest) * 100

75.0

In [27]:
ml.confusionMatrix(Ttrain, Ytrain, [1, 2]);

       1    2
    ------------
 1 |100.0  0  
 2 |  0  100.0


In [28]:
ml.confusionMatrix(Ttest, Ytest, [1, 2]);

       1    2
    ------------
 1 | 50.0 50.0
 2 |  0  100.0


Remember that linear logistic regression can be applied by specifying 0 hidden units.

In [29]:
netc = nn.NeuralNetworkClassifier(X.shape[1], 0, len(np.unique(T)))
netc.train(Xtrain, Ttrain, 100)
print(netc)
print('T, Predicted')
Ytrain = netc.use(Xtrain)
Ytest = netc.use(Xtest)

NeuralNetwork(1, [], 2)
   Network was trained for 101 iterations that took 0.1335 seconds. Final error is 1.3251306360973256e-05.
T, Predicted


In [30]:
ml.confusionMatrix(Ttrain, Ytrain, [1, 2]);

       1    2
    ------------
 1 |100.0  0  
 2 |  0  100.0


In [31]:
ml.confusionMatrix(Ttest, Ytest, [1, 2]);

       1    2
    ------------
 1 | 50.0 50.0
 2 |  0  100.0


## Apply to data from orthopedic patients

Download ```column_3C_weka.csv``` from [this Kaggle site](https://www.kaggle.com/uciml/biomechanical-features-of-orthopedic-patients).  Use the column named ```class``` to create your target class labels. Apply QDA, LDA, linear logistic regression, and nonlinear logistic regression to this data.  Experiment with different hidden layer structures and numbers of iterations and describe what you find.

Partition data into 80% for training and 20% for testing, with ```shuffle=True```.

Print percents of training and testing samples correctly classified by QDA, LDA and various neural network classifiers.  Also print confusion matrices for training and for testing samples for each classifier.  Discuss the relative performance of your classifiers.

## Grading and Check-in

Your notebook will be run and graded automatically. Test this grading process by first downloading [A4grader.tar](https://www.cs.colostate.edu/~anderson/cs445/notebooks/A4grader.tar) and extract A4grader.py from it.   

In [1]:
%run -i A4grader.py


======================= Code Execution =======================

========================RUNNING INSTRUCTORS SOLUTION!

Testing  import neuralnetworksA4 as nn

--- 5/5 points. The statement  import neuralnetworksA4 as nn  works.

Testing  import mlutilities as ml

--- 5/5 points. The statement  import mlutilities as ml  works.

Testing X = np.vstack((np.arange(20), [7, 4, 5, 5, 8, 4, 6, 7, 4, 9, 4, 2, 6, 6, 3, 3, 7, 2, 6, 4])).T
        T = np.array([1]*8 + [2]*8 + [3]*4).reshape((-1, 1))
        Xtrain, Ttrain, Xtest, Ttest = ml.partition(X, T, 0.8, classification=True, shuffle=False)

--- 10/10 points. ml.partition works correctly

Testing nnet = nn.NeuralNetworkClassifier(2, [5, 5], 3)

--- 10/10 points. nn.NeuralNetworkClassifier did not throw exception.

Testing nnet.train(Xtrain, Ttrain, 200)

--- 20/20 points. nnet.getErrors()[-1] is less than 1.e-5.

Testing Ytest = nnet.use(Xtest)
        fractionCorrect = np.sum(Ytest == Ttest) / len(Ttest)

--- 20/20 points. fractionCorrect 

# Extra Credit

Earn 1 extra credit point by doing a few experiments with different neural network classifiers using the ReLU activation function on the orthopedic data. Discuss any differences you see from your earlier results that used tanh.